In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE165004"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE165004"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE165004.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE165004.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE165004.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Endometrial Tissue RNA expression in recurrent pregnancy losses and unexplained infertility vs. conrol"
!Series_summary	"Recent studies are directed to decode the genetic signature of endometrial receptivity for better outcomes in assisted reproductive technologies. In this study, we aimed to understand the transcriptomic profile of midsecretory phase endometria of patients with recurrent pregnancy losses (RPL) and unexplained infertility (UI) by comparing with the endometria of healthy fertile women (Controls)."
!Series_summary	"In this prospective cohort study, we took endometrial samples from 24 patients with RPL, 24 patients with UI and 24 Controls at day 19-21 of the menstrual cycle. By performing genomic analysis, we assessed for differentially expressed genes (DEGs) and pathway analysis."
!Series_overall_design	"All patients involved in this prospective cohort study were recruited from Istanbul University School of Medicine between August 2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series title and summary, this dataset appears to contain RNA expression data
# from endometrial tissue, which indicates gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Checking the sample characteristics dictionary for trait, age, and gender information
# For trait (endometriosis), we can use row 0 which contains "subject status/group"
# Age and gender information are not explicitly available in the sample characteristics
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert trait values to binary (0 for control, 1 for disease).
    In this dataset, endometriosis status is inferred from the study group.
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert based on subject status
    if 'Control' in value:
        return 0  # Control group
    elif 'patient with RPL' in value or 'patient with UIF' in value:
        return 1  # Patient with condition (RPL or UIF)
    return None

def convert_age(value):
    """
    Convert age values to continuous.
    Not applicable for this dataset as age information is not available.
    """
    return None

def convert_gender(value):
    """
    Convert gender values to binary (0 for female, 1 for male).
    Not applicable for this dataset as gender information is not available.
    """
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only do this if trait data is available
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM5024320': [0.0], 'GSM5024321': [0.0], 'GSM5024322': [0.0], 'GSM5024323': [0.0], 'GSM5024324': [0.0], 'GSM5024325': [0.0], 'GSM5024326': [0.0], 'GSM5024327': [0.0], 'GSM5024328': [0.0], 'GSM5024329': [0.0], 'GSM5024330': [0.0], 'GSM5024331': [0.0], 'GSM5024332': [0.0], 'GSM5024333': [0.0], 'GSM5024334': [0.0], 'GSM5024335': [0.0], 'GSM5024336': [0.0], 'GSM5024337': [0.0], 'GSM5024338': [0.0], 'GSM5024339': [0.0], 'GSM5024340': [0.0], 'GSM5024341': [0.0], 'GSM5024342': [0.0], 'GSM5024343': [0.0], 'GSM5024344': [1.0], 'GSM5024345': [1.0], 'GSM5024346': [1.0], 'GSM5024347': [1.0], 'GSM5024348': [1.0], 'GSM5024349': [1.0], 'GSM5024350': [1.0], 'GSM5024351': [1.0], 'GSM5024352': [1.0], 'GSM5024353': [1.0], 'GSM5024354': [1.0], 'GSM5024355': [1.0], 'GSM5024356': [1.0], 'GSM5024357': [1.0], 'GSM5024358': [1.0], 'GSM5024359': [1.0], 'GSM5024360': [1.0], 'GSM5024361': [1.0], 'GSM5024362': [1.0], 'GSM5024363': [1.0], 'GSM5024364': [1.0], 'GSM5024365': [1.0], 'GSM502436

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 61
Header line: "ID_REF"	"GSM5024320"	"GSM5024321"	"GSM5024322"	"GSM5024323"	"GSM5024324"	"GSM5024325"	"GSM5024326"	"GSM5024327"	"GSM5024328"	"GSM5024329"	"GSM5024330"	"GSM5024331"	"GSM5024332"	"GSM5024333"	"GSM5024334"	"GSM5024335"	"GSM5024336"	"GSM5024337"	"GSM5024338"	"GSM5024339"	"GSM5024340"	"GSM5024341"	"GSM5024342"	"GSM5024343"	"GSM5024344"	"GSM5024345"	"GSM5024346"	"GSM5024347"	"GSM5024348"	"GSM5024349"	"GSM5024350"	"GSM5024351"	"GSM5024352"	"GSM5024353"	"GSM5024354"	"GSM5024355"	"GSM5024356"	"GSM5024357"	"GSM5024358"	"GSM5024359"	"GSM5024360"	"GSM5024361"	"GSM5024362"	"GSM5024363"	"GSM5024364"	"GSM5024365"	"GSM5024366"	"GSM5024367"	"GSM5024368"	"GSM5024369"	"GSM5024370"	"GSM5024371"	"GSM5024372"	"GSM5024373"	"GSM5024374"	"GSM5024375"	"GSM5024376"	"GSM5024377"	"GSM5024378"	"GSM5024379"	"GSM5024380"	"GSM5024381"	"GSM5024382"	"GSM5024383"	"GSM5024384"	"GSM5024385"	"GSM5024386"	"GSM5024387"	"GSM5024388"	"GSM5024389"	"GSM5024390"	"GSM5024391"
First data li

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown, I can see the expression data uses numeric probe IDs (1, 2, 3, etc.) 
# rather than standard human gene symbols like BRCA1, TP53, etc.
# These appear to be microarray probe IDs that need to be mapped to actual gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788'], 'CHROMOSOMAL_LOCATION': 

### Step 6: Gene Identifier Mapping

In [7]:
# Identify the appropriate columns for mapping
# From the preview, I can see:
# - 'ID' column contains numeric identifiers that match the gene expression data index
# - 'GENE_SYMBOL' column contains the human gene symbols we need

# 1. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 2. Apply the gene mapping to convert probe-level measurements to gene expression data
# Using the library function to handle the many-to-many mapping logic
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 3. Preview the result to verify
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First few genes: {list(gene_data.index[:5])}")
print(f"First few samples: {list(gene_data.columns[:5])}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data after mapping:
Shape: (20353, 72)
First few genes: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M']
First few samples: ['GSM5024320', 'GSM5024321', 'GSM5024322', 'GSM5024323', 'GSM5024324']


Gene expression data saved to ../../output/preprocess/Endometriosis/gene_data/GSE165004.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Endometriosis/gene_data/GSE165004.csv
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/GSE165004.csv
Linked data shape: (72, 19848)


For the feature 'Endometriosis', the least common label is '0.0' with 24 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Endometriosis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Endometriosis/GSE165004.csv
